# Chapter 15: The Sports Analytics case study

In [ ]:
import pandas as pd
import seaborn as sns
import json
from urllib import request

## Get the data

In [ ]:
shots_url = 'https://www.murach.com/python_analysis/shots.json'
shots = request.urlretrieve(shots_url, filename='shots.json')

In [ ]:
with open('shots.json') as jsonData:
    shots = json.load(jsonData)
shots.keys()

In [ ]:
columnHeaders = shots['resultSets'][0]['headers']
columnHeaders = [x.lower() for x in columnHeaders]
columnHeaders

In [ ]:
rows = shots['resultSets'][0]['rowSet']

In [ ]:
df = pd.DataFrame(data=rows, columns=columnHeaders)
df

## Clean the data

In [ ]:
df.period.unique()

In [ ]:
df = df.query('period < 5.0')
df

In [ ]:
df.nunique(dropna=False)

In [ ]:
shots = df.drop(columns=['grid_type','game_event_id','team_id',
    'team_name','player_id','shot_zone_range','shot_zone_basic',
    'shot_zone_area','event_type','action_type', 'minutes_remaining',
    'seconds_remaining', 'shot_distance','player_name','period','htm',
    'vtm','shot_attempted_flag'])

In [ ]:
shots.game_date = pd.to_datetime(shots.game_date)

In [ ]:
shots.info()

## Prepare the data

In [ ]:
shots.head(3)

In [ ]:
shots.set_index('game_id', inplace=True)
shots

In [ ]:
def get_season(row):
    if row.game_date.month > 6:
        season = f'{row.game_date.year}-{row.game_date.year + 1}'
    else:
        season = f'{row.game_date.year - 1}-{row.game_date.year}'
    return season

shots['season'] = shots.apply(get_season, axis=1)
shots.head()

In [ ]:
shots['shot_result'] = shots.shot_made_flag.replace({0:'Missed', 1:'Made'})
shots.head()

In [ ]:
shots['shot_type'].unique()

In [ ]:
shots['points_made'] = shots.apply(lambda x: 0 if x.shot_result == 'Missed' else 
                          (3 if x.shot_type == '3PT Field Goal' else 2), axis=1)
shots.head()

In [ ]:
shots['points_made_game'] = shots.groupby('game_id').points_made.transform('sum')
shots

In [ ]:
shots['shots_attempted'] = shots.groupby('game_id').shot_made_flag.transform('count')
shots

In [ ]:
shots['shots_made'] = shots.groupby('game_id').shot_made_flag.transform('sum')
shots

In [ ]:
shots[['shot_type','points_made','points_made_game','shots_attempted','shots_made']]
shots

## Plot the summary data

In [ ]:
shotsSeason = shots[['season','game_date','points_made_game','shots_made',
                     'shots_attempted']].drop_duplicates()
shotsSeason

In [ ]:
sns.catplot(data=shotsSeason, kind='box', x='season', y='points_made_game', 
            aspect=2.5, palette='deep')

In [ ]:
shotsSeason.head()

In [ ]:
shotsSeasonAvg = shotsSeason.groupby('season').mean().reset_index()
shotsSeasonAvg

In [ ]:
shotsSeasonAvg.plot(
    x='season', y=['points_made_game','shots_made','shots_attempted'], 
    color={'points_made_game':'red','shots_made':'blue','shots_attempted':'green'}, 
    figsize=(8,5), ylim=(0,30))

## Plot the shots for two games

In [ ]:
# two games with a lot of shots
gameIDs = ['0021800923','0021800642']
g = sns.relplot(data=shots.query('game_id in @gameIDs'), kind='scatter', 
    x='loc_x', y='loc_y', hue='shot_result', col='game_id')

In [ ]:
# SOURCE: http://savvastjortjoglou.com/nba-shot-sharts.html
from matplotlib.patches import Circle, Rectangle, Arc
def draw_court(ax=None, color='black', lw=2, outer_lines=False):
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = plt.gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30, -7.5), 60, -1, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc, center_outer_arc,
                      center_inner_arc]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax

In [ ]:
g = sns.relplot(data=shots.query('game_id in @gameIDs'), kind='scatter', 
    x='loc_x', y='loc_y', hue='shot_result', col='game_id', s=50)

for i, ax in enumerate(g.axes.flat):
    ax.set_title('shots for game ' + gameIDs[i])
    ax = draw_court(ax, outer_lines=True)
    ax.set_xlim(-300,300)
    ax.set_ylim(-100, 500)

## Plot shot data for two seasons

In [ ]:
colors = ['#FF0B04','#4374B3'] # blue and red
sns.set_palette(sns.color_palette(colors))

seasons = ['2009-2010','2018-2019']
g = sns.relplot(data=shots.query('season in @seasons'), kind='scatter', 
    x='loc_x', y='loc_y', hue='shot_result', col='season', col_wrap=1)

for ax in g.axes.flat:
    ax = draw_court(ax, outer_lines=True)
    ax.set_xlim(-300, 300)
    ax.set_ylim(-100, 500)

## Plot shot density for one season

In [ ]:
colors = ['#4374B3','#FF0B04'] # red and blue
sns.set_palette(sns.color_palette(colors))

g = sns.displot(data=shots.query('season == "2015-2016"'), kind='kde', legend=False,
    x='loc_x', y='loc_y', col='shot_result', hue='shot_result', col_wrap=1)

for ax in g.axes.flat:
    ax = draw_court(ax, outer_lines=True)
    ax.set_xlim(-300, 300)
    ax.set_ylim(-100, 500)

## Plot shot density for two seasons

In [ ]:
colors = ['#FF0B04','#4374B3'] # blue and red
sns.set_palette(sns.color_palette(colors))

seasons = ['2009-2010','2015-2016']
g = sns.displot(data=shots.query('season in @seasons'), kind='kde', 
                x='loc_x', y='loc_y', row='shot_result', col='season', 
                hue='shot_result', legend=False)
for ax in g.axes.flat:
    ax = draw_court(ax, outer_lines=True)
    ax.set_xlim(-300, 300)
    ax.set_ylim(-100, 500)